In [70]:
import torch
import re
from safetensors.torch import load_file

In [54]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [55]:
%%capture
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nogue\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nogue\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [56]:
from whoosh.index import open_dir
from whoosh.qparser import QueryParser, OrGroup
from transformers import AutoTokenizer, AutoModelForCausalLM

In [72]:
safetensors_path = 'ufam_legis.safetensors'
tokenizer_model_path = 'Autotrain\\autotrain-obceu-vsek1'

In [73]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_path, use_safetensors=True)
model_weights = load_file(safetensors_path)

In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = AutoModelForCausalLM.from_pretrained(model_name, state_dict=model_weights)

In [76]:
%%capture
model.to('cuda')

In [77]:
def pre_processar_query(query):
    stop_words = set(stopwords.words('portuguese'))
    word_tokens = word_tokenize(query.lower())
    filtered_query = [word for word in word_tokens if word not in stop_words]
    return " ".join(filtered_query)

In [78]:
def filtrar_paragrafos_por_palavras_chave(texto, palavras_chave):
    paragrafos = texto.split('\n')
    
    paragrafos_relevantes = []
    
    padrao_palavras_chave = re.compile(r'\b(?:' + '|'.join(re.escape(palavra) for palavra in palavras_chave) + r')\b', re.IGNORECASE)
    
    for paragrafo in paragrafos:
        if padrao_palavras_chave.search(paragrafo):
            paragrafos_relevantes.append(paragrafo)
    
    return paragrafos_relevantes

In [79]:
def gerar_resposta(query, index_dir, tokenizer, model, device='cuda', max_context_length=512):
    ix = open_dir(index_dir)

    query_tratada = pre_processar_query(query)
    
    with ix.searcher() as searcher:
        query_parser = QueryParser("content", ix.schema, group=OrGroup)
        parsed_query = query_parser.parse(query_tratada)
        print(f"Query processada: {parsed_query}")
        
        results = searcher.search(parsed_query, limit=4)
        retrieved_texts = [result['content'] for result in results]
        print(f"Textos recuperados: {retrieved_texts}")
    
    keywords = query_tratada.split()
    
    filtered_paragraphs = []
    for doc in retrieved_texts:
        filtered_paragraphs.extend(filtrar_paragrafos_por_palavras_chave(doc, keywords))
    
    context = "\n".join(filtered_paragraphs)
    
    if not context:
        return "Nenhum parágrafo relevante encontrado com as palavras-chave fornecidas."
    
    inputs = tokenizer(context, return_tensors="pt", truncation=True, max_length=max_context_length).to(device)
    
    input_text = f"Pergunta: {query}\nDocumentos relevantes: {context}\nResposta:"
    
    inputs = tokenizer(input_text, return_tensors='pt').to(device)

    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_new_tokens=200,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )
    
    new_tokens = outputs[0][inputs['input_ids'].shape[1]:]
    new_tokens_text = tokenizer.decode(new_tokens, skip_special_tokens=True)
    
    return new_tokens_text


In [80]:
query = "Quanto preciso tirar de nota para não fazer a prova final?"
index_dir = "indexdir"

In [81]:
response = gerar_resposta(query, index_dir, tokenizer, model)
print(response)

Query processada: (content:quanto OR content:preciso OR content:tirar OR content:nota OR content:fazer OR content:prova OR content:final OR content:?)
Textos recuperados: ['**Resolução Nº 038/2015 - CONSEPE**\n\n**Título**: Regulamenta a aceleração de estudos prevista no art. 71 do Regimento Geral da Universidade Federal do Amazonas e dá outras providências.\n\n**Considerando**:\n\n* O teor do Processo nº 275/2014 - CONSEPE;\n* O disposto no art. 47, 82º, da Lei 9.394/1996 (Lei de Diretrizes e Bases da Educação - LDB);\n* O Parecer CNE/CES nº 60/2007;\n* A necessidade de a UFAM regulamentar o art. 71 do Regimento Geral.\n\n**Artigos**:\n\n**CAPÍTULO I - DAS DISPOSIÇÕES GERAIS**\n\n* Art. 1º: O estudante regularmente matriculado na Universidade Federal do Amazonas poderá abreviar a duração do curso presencial, via aceleração dos estudos, observadas as disposições deste Regulamento.\n* Art. 2º: A aceleração de estudos poderá ser autorizada para todos os componentes curriculares previstos